In [30]:
import re
import nltk  
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

Helper code was provided by
https://www.geeksforgeeks.org/scrape-imdb-movie-rating-and-details-using-python/


1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8.   
a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.
b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.
c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  
d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  



In [31]:
#get the URL to all the thriller movies
url = 'https://www.imdb.com/list/ls095374765/?sort=moviemeter,asc&st_dt=&mode=grid&page=1&ref_=ttls_vw_smp&genres=Thriller'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [32]:
#empty lists to store the reivews
NP_Chunker = []
POS_Tagger = []

In [33]:
def list_Of_Reviews(website):
    soup = BeautifulSoup(website, 'html.parser')
    f = soup.find_all('div', class_='ipl-icon ipl-star-icon')
    f = soup.find_all('span', class_="rating-other-user-rating")

    idxPos = -1
    idxNeg = -1

    for idx, i in enumerate(f):
        sp = (f[idx].text.split('/'))
        sp = int(sp[0][-2:])
        if sp > 8:
            idxPos = idx
        elif sp < 4:
            idxNeg = idx
    return([idxNeg,idxPos])

In [34]:
for div in html_soup.findAll('div', attrs={'class':'title'}):
    if len(NP_Chunker) > 100:
        break
    else:
        url = 'https://www.imdb.com' + div.find('a')['href'] 
        url = url + 'reviews'
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        review_containers = html_soup.find_all('div', class_='text show-more__control')

        reviewIndex = list_Of_Reviews(response.content)
        reviewList = []

        if reviewIndex[0] == -1 or reviewIndex[1] == -1:
            pass
        else:
            reviewList.append(review_containers[reviewIndex[0]].text)
            reviewList.append(review_containers[reviewIndex[1]].text)
            count =+ 2

            for i in reviewList:
                text = nltk.word_tokenize(i)
                sentenceTag = nltk.pos_tag(text)
                POS_Tagger.append(sentenceTag)
                grammar = "NP: {<DT>?<JJ>*<NN>}"

                cp = nltk.RegexpParser(grammar)
                result = cp.parse(sentenceTag)
                NP_Chunker.append(result)

In [35]:
len(NP_Chunker)

100

In [36]:
print(NP_Chunker)

[Tree('S', [Tree('NP', [('The', 'DT'), ('movie', 'NN')]), ('was', 'VBD'), ('so', 'RB'), ('bad', 'JJ'), ('.', '.'), ('I', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('like', 'VB'), ('it', 'PRP'), ('at', 'IN'), ('all', 'DT'), (',', ','), ('but', 'CC'), ('frankly', 'RB'), ('I', 'PRP'), ('had', 'VBD'), Tree('NP', [('fun', 'NN')]), ('.', '.'), ('Tom', 'NNP'), ('Hardy', 'NNP'), ('suits', 'NNS'), Tree('NP', [('the', 'DT'), ('role', 'NN')]), ('very', 'RB'), ('well', 'RB'), ('.', '.'), ('I', 'PRP'), ('like', 'IN'), ('the', 'DT'), ('Carnage', 'NNP'), ('as', 'IN'), Tree('NP', [('character', 'NN')]), (',', ','), ('but', 'CC'), ('I', 'PRP'), ('wish', 'VBP'), ('it', 'PRP'), ('was', 'VBD'), Tree('NP', [('a', 'DT'), ('tougher', 'NN')]), ('and', 'CC'), Tree('NP', [('gore', 'NN')]), Tree('NP', [('movie', 'NN')]), ('.', '.'), Tree('NP', [('The', 'DT'), ('relationship', 'NN')]), ('between', 'IN'), ('Edin', 'NNP'), ('and', 'CC'), ('Venom', 'NNP'), ('were', 'VBD'), Tree('NP', [('like', 'JJ'), ('romantic', 'JJ')

2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:
a.	In Python, use BeautifulSoup to grab the main review text from each link.  
b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 
c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.



3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).

For this project I used the Thriller genre. The source of my reviews was IMDB and I used a defined function called list_Of_Reviews. I scraped the reviews of thriller movies from the first two pages of IMDB thriller since the first page only yeilds 98 reviews. I collected a total of 100 reviews via the scrapers and used a regex to determine what a NP is and pased it accordingly. The output was put into a single list.